# Introduction

In machine learning, hyperparameters are parameters that are not learned from data, but rather set prior to training. Selecting appropriate hyperparameters is crucial to achieving high accuracy in machine learning models. Two common methods for hyperparameter optimization are randomized search and grid search. In this lab, we will compare these two methods for optimizing hyperparameters of a linear Support Vector Machine (SVM) with Stochastic Gradient Descent (SGD) training.

# Import necessary libraries and load data

We will start by importing the necessary libraries and loading the digits dataset from scikit-learn.

In [1]:
import numpy as np
from time import time
import scipy.stats as stats
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier

# load digits dataset
X, y = load_digits(return_X_y=True, n_class=3)

# Create an SVM model

We will create a linear SVM model with SGD training.

In [2]:
# create SVM model with SGD training
clf = SGDClassifier(loss="hinge", penalty="elasticnet", fit_intercept=True)

# Randomized search for hyperparameter optimization

We will use randomized search to explore the hyperparameter space and find the best hyperparameters for our SVM model.

In [4]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")

# specify parameters and distributions to sample from
param_dist = {
    "average": [True, False],
    "l1_ratio": stats.uniform(0, 1),
    "alpha": stats.loguniform(1e-2, 1e0),
}

# run randomized search
n_iter_search = 15
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=n_iter_search
)

start = time()
random_search.fit(X, y)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)

# print results
report(random_search.cv_results_)

RandomizedSearchCV took 0.54 seconds for 15 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.987 (std: 0.013)
Parameters: {'alpha': 0.02689792231956257, 'average': True, 'l1_ratio': 0.9137494958416262}

Model with rank: 2
Mean validation score: 0.983 (std: 0.021)
Parameters: {'alpha': 0.06465372942736058, 'average': False, 'l1_ratio': 0.8395862399561652}

Model with rank: 3
Mean validation score: 0.979 (std: 0.019)
Parameters: {'alpha': 0.011087822554910951, 'average': False, 'l1_ratio': 0.2191086930071391}



# Grid search for hyperparameter optimization

We will use grid search to explore the hyperparameter space and find the best hyperparameters for our SVM model.

In [5]:
# specify parameters to search over
param_grid = {
    "average": [True, False],
    "l1_ratio": np.linspace(0, 1, num=10),
    "alpha": np.power(10, np.arange(-2, 1, dtype=float)),
}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)

start = time()
grid_search.fit(X, y)

print(
    "GridSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(grid_search.cv_results_["params"]))
)

# print results
report(grid_search.cv_results_)

GridSearchCV took 1.58 seconds for 60 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.994 (std: 0.007)
Parameters: {'alpha': 0.1, 'average': False, 'l1_ratio': 0.2222222222222222}

Model with rank: 2
Mean validation score: 0.991 (std: 0.010)
Parameters: {'alpha': 0.01, 'average': False, 'l1_ratio': 0.1111111111111111}

Model with rank: 3
Mean validation score: 0.989 (std: 0.007)
Parameters: {'alpha': 0.01, 'average': False, 'l1_ratio': 0.5555555555555556}

Model with rank: 3
Mean validation score: 0.989 (std: 0.007)
Parameters: {'alpha': 1.0, 'average': False, 'l1_ratio': 0.0}



# Summary

In this lab, we compared randomized search and grid search for hyperparameter optimization of a linear SVM model with SGD training. We found that both methods explored the same hyperparameter space, but randomized search was significantly faster. The best hyperparameters found by each method were similar in performance, but randomized search may have slightly worse performance due to noise. In practice, we would not search over so many hyperparameters simultaneously, but only the most important ones.